In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [8]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1434, with_units=False)
Hist_table


Years,Entries
IntegerAndSexagesimal,Historical
365 ;,"04s 09 ; 23,02,37"
730 ;,"08s 18 ; 46,05,15"
1095 ;,"28 ; 09,07,52"
1461 ;,"05s 20 ; 42,54,31"
1826 ;,"10s 00 ; 05,47,08"
2191 ;,"02s 09 ; 28,50,45"
2556 ;,"06s 18 ; 51,53,23"
2922 ;,"11s 11 ; 25,30,01"
3287 ;,"03s 20 ; 48,33,29"


In [9]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(34):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [11]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("13;10,35,1,15,11,4,35"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(34):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=7):
        days=Hist_table[i][0]
        Test_recomp[i][1]=(daily_mean*days)%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=3):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=60*60*60*(Hist_table[i][1]-Test_recomp[i][1])


In [12]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(-191636.9705882353, nan)

In [13]:
B.astype(float).style.background_gradient(axis=None)

,Entries
Years,
365 ;,0.000000
730 ;,0.000000
1095 ;,0.000000
1461 ;,540.000000
1826 ;,-60.000000
2191 ;,0.000000
2556 ;,0.000000
2922 ;,-60.000000
3287 ;,-10.000000


In [14]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Lunar mean years.csv')
B.to_csv('Lunar mean years.zip', index=False, compression=compression_opts)